In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/?sortby=post_date%3ADesc&pagesize=20'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    for name, rating in zip(names[1:], ratings):
        review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'ana all nippon airways'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/ana.csv',index=False)

In [4]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,ana all nippon airways,Larry Silverstein,2020-04-11,✅ Trip Verified | New York to Tokyo with ANA....,Couple Leisure,Business Class,10,5,5,5,4,5,5,yes
1,ana all nippon airways,Ken Inaba,2020-04-01,✅ Trip Verified | Fukuoka to Vancouver via T...,Solo Leisure,Economy Class,9,4,5,5,5,4,5,yes
2,ana all nippon airways,Michelle Massey,2020-03-25,✅ Trip Verified | Narita to Perth. ANA kept us...,Couple Leisure,Business Class,10,5,5,5,5,5,5,yes
3,ana all nippon airways,Jarosław Miazga,2020-02-25,✅ Trip Verified | A very enjoyable flight. I ...,Solo Leisure,Economy Class,10,4,5,5,5,5,5,yes
4,ana all nippon airways,Mohamed Elsamak,2020-02-24,✅ Trip Verified | Kuala Lumpur to Los Angeles...,Solo Leisure,Economy Class,10,5,5,5,5,5,5,yes
